In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train, get_squad_train_chars
from itertools import chain

%aimport QANet_glove

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
%%time
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/'

data = np.load(squad_path + 'train_with_glove_vocab.pkl')
data_char = np.load(squad_path + 'train_char_ascii.pkl')
glove_embs = np.load(glove_path + 'glove.6B.300d.npy')
voc_size = glove_embs.shape[0]
alphabet_size = 128

# Some answers get broken in the process of tokenization, because some answer words are not properly split.
def filter_broken_answers(data, data_char):
    data_new = []
    data_char_new = []
    for i in xrange(len(data)):
        if data[i][0]:
            data_new.append(data[i])
            data_char_new.append(data_char[i])
    return data_new, data_char_new

print 'Examples total:', len(data), len(data_char)

data, data_char = filter_broken_answers(data, data_char)

print 'Working examples:', len(data), len(data_char)

data = (data, data_char)

Examples total: 87599 87599
Working examples: 86474 86474
CPU times: user 1min 5s, sys: 1.72 s, total: 1min 6s
Wall time: 1min 11s


In [3]:
trim = 300
data = zip(*[(d0[:2] + [d0[2][:trim]], [d1[0], d1[1][:trim]]) for d0, d1 in zip(*data) if max(d0[0][0]) < trim])
print 'Trimmed examples:', len(data[0])

Trimmed examples: 86355


In [4]:
%%time
data_dev = np.load(squad_path + 'dev_with_glove_vocab.pkl')
data_dev_char = np.load(squad_path + 'dev_char_ascii.pkl')

CPU times: user 6.09 s, sys: 71.7 ms, total: 6.16 s
Wall time: 6.2 s


## QANet

In [33]:
%%time
update_fn = lambda l, p: L.updates.adam(l, p)

qa_net = QANet_glove.QANet(voc_size=voc_size,
                           alphabet_size=alphabet_size,
                           emb_size=300,
                           emb_char_size=20,
                           num_emb_char_filters=200,
                           rec_size=300,
                           emb_init=glove_embs,
                           train_inds=[],
                           update_fn=update_fn)

Building the model...
Using custom update_fn.
Compiling theano functions:
    train_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 1min 7s, sys: 1.77 s, total: 1min 8s
Wall time: 1min 10s


In [29]:
# qa_net.load_params('trained_models/glove_unks/charemb_glove_train_unk_dropout_ep5.npz', glove_embs)

In [ ]:
# z char-embeddings

# 4 ep

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# słownik glove, unk to średnie słowo, nie trenujemy zanurzeń słów, char-embeddings ma losowy init

qa_net.train_one_epoch(data, 15, log_interval=5)

Done 5 batches in 5.83s	training loss:	1.783775
Done 10 batches in 10.13s	training loss:	2.065273
Done 15 batches in 14.16s	training loss:	2.066628
Done 20 batches in 18.54s	training loss:	2.090096
Done 25 batches in 22.42s	training loss:	2.140420
Done 30 batches in 26.83s	training loss:	2.112451
Done 35 batches in 30.81s	training loss:	2.115174
Done 40 batches in 34.97s	training loss:	2.103173
Done 45 batches in 39.19s	training loss:	2.075913
Done 50 batches in 43.78s	training loss:	2.121864
Done 55 batches in 48.20s	training loss:	2.114348
Done 60 batches in 52.63s	training loss:	2.102968
Done 65 batches in 56.97s	training loss:	2.125282
Done 70 batches in 61.10s	training loss:	2.104288
Done 75 batches in 65.12s	training loss:	2.119227
Done 80 batches in 69.31s	training loss:	2.125396
Done 85 batches in 73.72s	training loss:	2.121410
Done 90 batches in 77.68s	training loss:	2.105622
Done 95 batches in 81.90s	training loss:	2.120466
Done 100 batches in 86.32s	training loss:	2.111490
D

In [ ]:
qa_net.save_params('trained_models/glove_vocab/charemb_all_fixed_ep4')

### QANet tests

In [ ]:
'''
charemb all_fixed 2ep {"f1": 63.93851222573222, "exact_match": 53.33964049195837} 
charemb all_fixed 3ep {"f1": 64.65644340794375, "exact_match": 54.71144749290445}

'''

In [47]:
def predict_spans(data, beam=10, batch_size=10):
    num_examples = len(data[0])
    
    start_probs = qa_net.get_start_probs(data, batch_size)
    best_starts = start_probs.argpartition(-beam, axis=1)[:, -beam:].astype(np.int32)
    
    scores = start_probs[np.arange(num_examples)[:, np.newaxis], best_starts]
    scores = np.tile(scores[:, np.newaxis], (beam, 1)).transpose(0, 2, 1)
    
    best_ends_all = []
    for i in xrange(beam):
        end_probs = qa_net.get_end_probs(data, best_starts[:, i], batch_size)
        best_ends = end_probs.argpartition(-beam, axis=1)[:, -beam:]
        scores[:, i, :] *= end_probs[np.arange(num_examples)[:, np.newaxis], best_ends]
        best_ends_all.append(best_ends)
        
    best_ends_all = np.hstack(best_ends_all)
        
    scores = scores.reshape(num_examples, beam**2)
    best_spans = scores.argmax(axis=1)
    starts = [i / beam for i in best_spans]
    
    starts = best_starts[np.arange(num_examples), starts]
    ends = best_ends_all[np.arange(num_examples), best_spans]
    
    return starts, ends

In [48]:
%%time
predicted_spans = []
batch_size = 10

idx = 0
while idx < len(data_dev):
    spans = predict_spans((data_dev[idx:idx + batch_size], data_dev_char[idx:idx + batch_size]), beam=1)
    predicted_spans.append(np.vstack(spans))
    idx += batch_size
    if not idx % 100:
        print idx, 'examples'
        
print 'Predictions done'
    
predicted_spans = np.hstack(predicted_spans).T

100 examples
200 examples
300 examples
400 examples
500 examples
600 examples
700 examples
800 examples
900 examples
1000 examples
1100 examples
1200 examples
1300 examples
1400 examples
1500 examples
1600 examples
1700 examples
1800 examples
1900 examples
2000 examples
2100 examples
2200 examples
2300 examples
2400 examples
2500 examples
2600 examples
2700 examples
2800 examples
2900 examples
3000 examples
3100 examples
3200 examples
3300 examples
3400 examples
3500 examples
3600 examples
3700 examples
3800 examples
3900 examples
4000 examples
4100 examples
4200 examples
4300 examples
4400 examples
4500 examples
4600 examples
4700 examples
4800 examples
4900 examples
5000 examples
5100 examples
5200 examples
5300 examples
5400 examples
5500 examples
5600 examples
5700 examples
5800 examples
5900 examples
6000 examples
6100 examples
6200 examples
6300 examples
6400 examples
6500 examples
6600 examples
6700 examples
6800 examples
6900 examples
7000 examples
7100 examples
7200 examples
7

In [49]:
np.savez('evaluate/glove_vocab/dev_with_glove_vocab_predictions_charemb_all_fixed_ep3.npz', predicted_spans)